<a href="https://colab.research.google.com/github/Tahimi/convectionDiffusion/blob/main/animated1DConvDiff_analyticalSolution2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analytical solution of Convection-Diffusion Equation in 1D with Dirichlet BCs

In [1]:
# mounting the drive
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# space discretization
import numpy as np

xL = 0.0
delX = 1.0
xR = xL + delX
nx = 100
dx = delX/nx
x = np.linspace(xL, xR, nx+1)

print('xL = x[0] = ', x[0], '\n'
      'xR = x[-1] = ', x[-1], '\n'
      'dx = ', dx, '\n'
      'nx = ', nx)

xL = x[0] =  0.0 
xR = x[-1] =  1.0 
dx =  0.01 
nx =  100


In [3]:
# time discretization

tL = 0.0
delT = 0.2
tR = tL + delT
dt = 0.001
nt = int(delT/dt)
t = np.linspace(tL, tR, nt+1)

print('tL = t[0] = ', t[0], '\n'
      'tR = t[-1] = ', t[-1], '\n'
      'dt = ', dt, '\n'
      'nt = ', nt)

tL = t[0] =  0.0 
tR = t[-1] =  0.2 
dt =  0.001 
nt =  200


In [4]:
# time instants for the saving of y
import numpy as np

delIdx = int(len(t)/4) # saving frequency
save_times = t[range(0, len(t), delIdx)]

closest_idx = lambda val, arr: np.abs(arr - val).argmin()
save_indices = [closest_idx(save_time, t) for save_time in save_times]

print('save_times = ', save_times, '\n'
      'save_indices = ', save_indices)

save_times =  [0.   0.05 0.1  0.15 0.2 ] 
save_indices =  [0, 50, 100, 150, 200]


In [5]:
# setting diffusion and covection parameters

# diffusion coefficient
alpha = 1

# advection velocity
beta = 5

In [6]:
# computing eigenvalues

n = 70
a = np.empty(n)
N = np.arange(1,n+1,1)
for i in N:
    a[i-1]=(-(i*np.pi/delX)**2)

In [7]:
import scipy.integrate

# unkown's initial distribution
f = lambda x: np.sin(np.pi*x/delX)

# analytical solution
def u(x, t, beta):
    sum = 0
    for i in N:
        sum = sum+2/xR*scipy.integrate.quad(lambda x:np.exp(-beta/(2*alpha**2)*x)*f(x)* \
                                            np.sin(i*np.pi*x/xR),0,xR)[0]*np.sin(i*np.pi*x/xR)* \
                                            np.exp(-alpha**2*(i*np.pi/xR)**2*t)
    return sum*np.exp(-beta**2/(4*alpha**2)*t)*np.exp(beta/(2*alpha**2)*x)

In [8]:
# Loop over time, calculate y and save it at save_times
import pandas as pd

df = pd.DataFrame(data={'x':x})
tiSTR = lambda ti: str(round(ti,3))
for i, ti in enumerate(t):
    y = u(x, ti, beta)
    if i in save_indices:
        df[f'y_{tiSTR(ti)}'] = y
df.head(3)

,x,y_0.0,y_0.05,y_0.1,y_0.15,y_0.2
0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.01,0.031404,0.005502,0.002130,0.000919,0.000407
2,0.02,0.062818,0.011275,0.004364,0.001884,0.000835


In [9]:
# producing the animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# define plot style
plt.style.use("ggplot")

# Create the figure and axis
fig, ax = plt.subplots()

# Initialize the line plot
ti = tL
lineInit, = ax.plot(x, u(x,ti,beta), color='r', label='yInit')

# set axis labels
ax.set_xlabel('x')
ax.set_ylabel('y')

# set figure bounding box
offset = 0.1
ax.set_xbound(xL-offset,xR+offset)
ax.set_ybound(0-offset,1.+offset)

# set axis ticks
plt.xticks(np.arange(xL, xR+1e-10, delX/10))
plt.yticks(np.arange(0., 1.+1e-10, (1.-0.)/10))

# initialize line (y values) and plotTitle (the figure's title)
line, = ax.plot(x, u(x,ti,beta), color='b', linestyle='dashed', label='y')
plotTitle = ax.set_title("t = " + tiSTR(tL))

# adding the legend
ax.legend()

# close the figure (to show only the animation)
plt.close(fig)

# Update line and plotTitle for the animation
def update(time_idx):
    ti = tL + time_idx*dt
    y = df[f'y_{tiSTR(ti)}']
    line.set_ydata(y)
    plotTitle.set_text('t = ' + tiSTR(ti))
    return line,

# Set up the animation
ani = FuncAnimation(fig, update, frames=save_indices, interval=200, blit=False)

# Save the animation as a GIF
gifFileName = '/content/drive/MyDrive/UFAL/advection_diffusion/FiniteDiference_FipyFiniteVolume/animated1DConvDiffDirichletBCs.gif'
ani.save(gifFileName, writer='pillow')

# Display the animation
HTML(ani.to_jshtml())